In [ ]:
import spacy
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
scholars = np.load("/content/drive/MyDrive/AI_Scholar_gender/78k_scholar_new.npy", allow_pickle=True)

Mounted at /content/drive


In [ ]:
organizations = []
organization_codes = []
for each in scholars:
  organization = BeautifulSoup(each["extra_info"][0], 'html.parser').text
  try:
    organization_code = each["extra_info"][0].split("org=")[1].split('"')[0]
  except:
    organization_code = None
  organizations.append(organization)
  organization_codes.append(organization_code)
df = pd.DataFrame({'organization': organizations, 'organization_code': organization_codes})

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.researchgate.net/profile/Jeffrey_Howard2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://github.com/puolival" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336:

In [ ]:
df_none = df[df["organization_code"].isnull()]

In [ ]:
df = df.dropna()

In [ ]:
organization2occur = {}
for index, row in df.iterrows():
  if row["organization_code"] not in organization2occur:
    organization2occur[row["organization_code"]] = [row]
  else:
    organization2occur[row["organization_code"]].append(row)

In [ ]:
organization = []
organization_code = []
occurance = []
for each in organization2occur:
  occurance.append(len(organization2occur[each]))
  organization.append(organization2occur[each][0][0])
  organization_code.append(organization2occur[each][0][1])
df = pd.DataFrame({'organization': organization, 'organization_code': organization_code, 'occurance': occurance})

In [ ]:
df = df.sort_values('occurance', ascending=False)
# df_new = df[df["occurance"] > 10]
len(df)

3568

In [ ]:
! pip install spacy
! pip install nltk
! python -m spacy download en_core_web_sm
  
# imports and load spacy english language package
import spacy
from spacy import displacy
from spacy import tokenizer
nlp = spacy.load('en_core_web_sm')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 31.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
org_ents = []
for index, row in df_none.iterrows():
  text = nlp(row["organization"])
  ents = [(e.text, e.label_) for e in text.ents]
  org_ent = ""
  for each in ents:
    if each[1] == 'ORG':
      org_ent += each[0]
  org_ent = (org_ent if org_ent != "" else None)
  org_ents.append(org_ent)
df_none["org_ent"] = org_ents

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
df_none_new = df_none[df_none["org_ent"].notnull()]

,organization,organization_code,org_ent
14,"Chief Technology Officer, Zensar Technologies ...",None,Zensar Technologies Ltd.
19,Microsoft Turing,None,Microsoft
21,PhD Scholar. University of Science and Technol...,None,University of Science and Technology Bannu
23,Research Scientist at Facebook AI,None,Facebook AI
31,"Principal Data Scientist at Bayer AG, Decision...",None,Bayer AGDecision Science Pharma
...,...,...,...
78040,Professor of Digital Linguistics and Writing R...,None,Digital Linguistics and Writing ResearchZHAW Z...
78049,Meisei University,None,Meisei University
78054,"Postdoc at CSAIL, MIT",None,CSAILMIT
78055,Al-Imam Muhammad Bin Saud Islamic University (...,None,IMSIU


In [ ]:
# Utility function for generating sentence embedding from the text
def get_embeddinngs(text):
    return nlp(text).vector

# Generating sentence embedding from the text
df_none_new['emb'] = df_none_new['organization'].apply(get_embeddinngs)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from nltk.cluster import KMeansClusterer
import nltk
def clustering_question(data,NUM_CLUSTERS = 25):

    sentences = data['org_ent']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters
clustering_question(df_none_new)

In [ ]:
! pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 4.2 MB 49.4 MB/s 
     |████████████████████████████████| 1.2 MB 57.6 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
     |████████████████████████████████| 596 kB 67.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=146c2d6658ff18341f0c181ee0e26fc82b81bf4605eb9c0d35aa28751d873bcc
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
import time
org_ent = list(df_none_new["org_ent"].values)
corpus_embeddings = model.encode(org_ent, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=10, threshold=0.75)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Batches:   0%|          | 0/286 [00:00<?, ?it/s]

Start clustering
Clustering done after 1.88 sec


In [ ]:
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", org_ent[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", org_ent[sentence_id])


Cluster 1, #277 Elements 
	 IBM Research
	 IBM Research
	 IBM Research
	 ...
	 IBM Research - Haifa
	 IBM ResearchIndian Institute of Technology DelhiIndian Institute of Technology
	 IBM ResearchHaifa Labs

Cluster 2, #267 Elements 
	 Amazon.com Services LLC
	 Amazon.com Services LLC
	 Amazon.com Services, Inc.
	 ...
	 Amazon
	 Amazon
	 Amazon

Cluster 3, #230 Elements 
	 Microsoft Research
	 Microsoft Research
	 Microsoft Research
	 ...
	 Microsoft Research Redmond
	 Microsoft ResearchUniversity of Washington
	 Microsoft ResearchRedmond

Cluster 4, #212 Elements 
	 Department of Computer Science& EngineeringAmity University
	 Department of Computer ScienceTaif University
	 Department of Computer ScienceUtah State University
	 ...
	 School of Computer Sciences
	 Department of Computer EngineeringMuthiah Polytechnic CollegeAnnamalainagar
	 School of Electronic Engineering and Computer Science

Cluster 5, #159 Elements 
	 University of Chinese
	 Chinese University of Hong Kong
	 Chinese